In [1]:
import numpy as np
import torch
from model import bertclassify, text_decoder

my_model = bertclassify(1)
my_model=my_model.cuda()

my_models=[text_decoder().cuda(),
           text_decoder().cuda(),
           text_decoder().cuda(),
           text_decoder().cuda(),
           text_decoder().cuda(),
           text_decoder().cuda(),
           text_decoder().cuda(),
           text_decoder().cuda()]
for index,onemodel in enumerate(my_models):
    if index<4:
        onemodel.load_state_dict(torch.load("model/new/decoder_%d.pth"%index))
    else:
        onemodel.load_state_dict(torch.load("model/new/last-4/decoder_%d.pth"%(index-4)))
my_model.load_state_dict(torch.load("model/new/encoder.pth"))


<All keys matched successfully>

In [11]:
import numpy as np
def myeval(pred, tar):
    pred_zero = torch.zeros_like(pred)
    pred_zero[pred > 0.5] = 1
    judge_n=pred_zero.cuda()==tar.cuda()
    error_num=0
    for index,i in enumerate(judge_n):
        if False == i[0]:
            error_num+=1
    return error_num,len(judge_n),1-error_num/len(judge_n)

from torch.utils.data import DataLoader
from dataset import TextDataset
numClass=8
train_dataset = TextDataset(True)
train_data_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
allRightnum=[]
allknum=[]
for i in range(numClass):
    allknum.append(0)
    allRightnum.append(0)
allknum=np.asarray(allknum)
allRightnum=np.asarray(allRightnum)
for input_data in train_data_loader:
    de_onemodel = my_model(input_data)

    for idclass in range(numClass):
        my_models[idclass].eval()
        output = my_models[idclass](de_onemodel)
        tar=torch.unsqueeze(input_data['label'][:,idclass].cuda(),1)
        errornum,knum,acc = myeval(output, tar)
        allRightnum[idclass]+=knum-errornum
        allknum[idclass]+=knum
        # print("%drecall:"%idclass,(sum(tar)-errornum)/sum(tar))
    print("allrecall",allRightnum/allknum)
    print("allnum:",allknum," allrightnum:",allRightnum)

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000019A5ECF3340>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 7dfc641f-2eb0-444e-af70-ae63a4325110)')' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve/main/tokenizer_config.json
12/10/2023 23:44:20 - WARNING - huggingface_hub.utils._http -   '(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-chinese/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000019A5ECF3340>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 7dfc641f-2eb0-444e-af70-ae63a4325110)')' thrown while requesting HEAD https://huggingface.co/bert-base-chinese/resolve


maxlen: 162
allrecall [0.90625 0.9375  0.84375 0.90625 1.      0.96875 1.      0.9375 ]
allnum: [32 32 32 32 32 32 32 32]  allrightnum: [29 30 27 29 32 31 32 30]
allrecall [0.859375 0.953125 0.84375  0.90625  0.984375 0.96875  1.       0.921875]
allnum: [64 64 64 64 64 64 64 64]  allrightnum: [55 61 54 58 63 62 64 59]
allrecall [0.86458333 0.95833333 0.82291667 0.90625    0.97916667 0.97916667
 1.         0.9375    ]
allnum: [96 96 96 96 96 96 96 96]  allrightnum: [83 92 79 87 94 94 96 90]
allrecall [0.859375  0.9609375 0.8203125 0.90625   0.9765625 0.96875   1.
 0.9375   ]
allnum: [128 128 128 128 128 128 128 128]  allrightnum: [110 123 105 116 125 124 128 120]
allrecall [0.86875 0.96875 0.83125 0.9125  0.98125 0.975   0.99375 0.925  ]
allnum: [160 160 160 160 160 160 160 160]  allrightnum: [139 155 133 146 157 156 159 148]
allrecall [0.875      0.953125   0.83854167 0.91666667 0.97916667 0.97395833
 0.99479167 0.92708333]
allnum: [192 192 192 192 192 192 192 192]  allrightnum: [168 

In [4]:
from dataset import test_TextDataset
import csv
from torch.utils.data.dataloader import DataLoader
label_list = {0:'Love', 1:'Joy', 2:'Anxiety', 3:'Sorrow', 4:'Expect', 5:'Hate', 6:'Surprise', 7:'Anger'}
pred=[[1,0,0,0,0,0,0,0]]

test_dataset = test_TextDataset()
test_data_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False)
alllable=[]
for input_data in test_data_loader:
    de_onemodel = my_model(input_data)
    lable=np.array([[]])

    for idclass in range(8):
        my_models[idclass].eval()
        output = my_models[idclass](de_onemodel)
        pred_tar=torch.zeros_like(output)
        pred_tar[output > 0.1] = 1
        onelable=pred_tar.cpu().detach().numpy()
        if idclass==0:
            lable=onelable
        else:
            lable=np.append(lable,onelable,axis=1)

    for onelable in lable:
        textlable=[]
        for index,l in enumerate(onelable):
            if l==1:
                textlable.append(label_list.get(index))
        alllable.append(textlable)
        pass


maxlen: 153


In [5]:
with open("data/test_pred.csv","w",newline="") as f:
    writer=csv.writer(f)
    writer.writerow(["ID","Label"])
    for i in range(len(alllable)):
        writer.writerow([i+1,alllable[i]])